### Download data

- Lower file size from 17GB to 1.5GB
    - Remove some unnecessary data
    - Compress as xz 

In [15]:
# !wget https://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/goa_uniprot_all.gaf.gz  -O goa_uniprot_all_ebi.gaf.gz.download
# !mv goa_uniprot_all_ebi.gaf.gz.download goa_uniprot_all_ebi.gaf.gz
# !gunzip -c goa_uniprot_all_ebi.gaf.gz | awk 'BEGIN {OFS="\t";FS="\t"} ($1 == "UniProtKB") {print $2,$4,$5,$7,$9,$14}' | sort -u | xz -T0 > goa_uniprot_all_ebi_filtered.tsv.xz.tmp
# !mv goa_uniprot_all_ebi_filtered.tsv.xz.tmp ../data/raw/gene_ontology/goa_uniprot_all_ebi_filtered.tsv.xz

In [ ]:
# TODO oneliner, did it work now?

# wget https://ftp.ebi.ac.uk/pub/databases/GO/goa/UNIPROT/goa_uniprot_all.gaf.gz -O - | gunzip -c | awk 'BEGIN {OFS="\t";FS="\t"} ($1 == "UniProtKB") {print $2,$4,$5,$7,$9,$14}' | sort -u | xz -T0 > goa_uniprot_all_ebi_filtered.tsv.xz

### Read into pandas

Using optimized dtypes to save space

In [17]:
import pandas as pd
from subpred.util import save_df
DATASET_FOLDER = "../data/datasets"

In [18]:
df_go_ebi = pd.read_table(
    "../data/raw/gene_ontology/goa_uniprot_all_ebi_filtered.tsv.xz",
    # force_update=True,
    header=None,
    names=["Uniprot", "qualifier", "go_id", "evidence_code", "aspect", "date"],
    dtype={
        "Uniprot": "string",
        "qualifier": "category",
        "go_id": "string",
        "evidence_code": "category",
        "aspect": "category",
        "date": "int",
    },
    # parse_dates=["date"],
)


In [ ]:
print(df_go_ebi.shape)
df_go_ebi = df_go_ebi.drop("date", axis=1)
df_go_ebi = df_go_ebi.drop_duplicates()
print(df_go_ebi.shape)

In [31]:
df_go_ebi = df_go_ebi.reset_index(drop=True)

### Save pickles containing datasets:


In [33]:
save_df(df_go_ebi, "go_all", DATASET_FOLDER)

Filtered dataframe with no IEA terms

In [27]:
df_go_ebi_mf_enables = df_go_ebi[
    (df_go_ebi.qualifier == "enables")&
    (df_go_ebi.aspect == "F")
].drop(["qualifier", "aspect"], axis=1).reset_index(drop=True)
save_df(df_go_ebi_mf_enables, "go_mf", DATASET_FOLDER)

In [28]:
from subpred.util import load_df

load_df("go_mf", DATASET_FOLDER)

,Uniprot,go_id,evidence_code
0,A0A000,GO:0003824,IEA
1,A0A000,GO:0003870,IEA
2,A0A000,GO:0030170,IEA
3,A0A001,GO:0000166,IEA
4,A0A001,GO:0005524,IEA
...,...,...,...
351680369,Z9JZ78,GO:0050048,IEA
351680370,Z9JZ78,GO:0052654,IEA
351680371,Z9JZ78,GO:0052655,IEA
351680372,Z9JZ78,GO:0052656,IEA


In [34]:
!du -hs ../data/datasets/*

9.9G	../data/datasets/go_all.pickle
4.6G	../data/datasets/go_mf.pickle
656K	../data/datasets/interpro.pickle
40K	../data/datasets/keywords.pickle
158M	../data/datasets/uniprot70.pickle
498M	../data/datasets/uniprot.pickle


In [23]:
# df_go_ebi_filtered = df_go_ebi[df_go_ebi.Uniprot.isin(df_uniprot.index)]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.qualifier == "enables"]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.evidence_code != "IEA"]
# df_go_ebi_filtered = df_go_ebi_filtered[df_go_ebi_filtered.aspect == "F"]
# df_go_ebi_filtered = df_go_ebi_filtered.drop(["qualifier", "aspect", "date"], axis=1).reset_index(drop=True)